In [ ]:
import os
from datetime import datetime

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

output_file_path = 'new_plan.xlsx'

# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = r'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')

env_file_path = 'D:/Projects/.env'
load_dotenv(env_file_path)
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_ASKGLOBAL")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)

start_date = datetime(2024, 2, 1).strftime('%Y%m%d')
tomorrow_date = datetime(2024, 3, 31).strftime('%Y%m%d')

sql_query: str = """
SELECT O.Name                    as Base,
       D.Name                    AS DocKind,
       i.Number                  AS InvoiceNumber,
       G.GoodId                  as Goodid,
       G.Name                    AS GoodName,
       M.Name                    AS Producer,
       C.Inn                     as Inn,
       C.FindName                AS ClientName,
       PIM.Name                  AS InvoiceManager,
       PCM.Name                  AS ClientManager,
       I.PaymentTerm,
       il.pBasePrice             as BasePrice,
       il.Price                  as SellingPrice,
       il.Kolich                 AS Quantity,
       I.DataEntered            AS DataEntered,
       il.pBasePrice * il.Kolich AS BaseAmount,
       il.pSumma                 AS TotalAmount
FROM INVOICELN il
         JOIN INVOICE i ON il.InvoiceId = i.InvoiceId
         JOIN PERSONAL PIM ON i.PersonalId = PIM.PersonalId
         JOIN DOCKIND D ON i.DocKindId = D.DocKindId
         JOIN CLIENT C ON i.ClientId = C.ClientId
         JOIN INCOMELN incl ON il.IncomeLnId = incl.IncomeLnId
         JOIN Good G ON incl.GoodId = G.GoodId
         JOIN Producer M ON M.ProducerId = G.ProducerId
         JOIN PERSONAL PCM ON C.PersonalId = PCM.PersonalId
         join OFFICE O on C.OfficeId = O.OfficeId

WHERE YEAR(I.DataEntered) = 2024
  and MONTH(I.DataEntered) = 3
  and D.name IN ('Финансовая скидка', 'Оптовая реализация', 'Возврат товара от покупателя')
order by i.DataEntered desc;
"""

#####################  EXECUTION  ######################
df = pd.read_sql_query(sql_query, engine)
df['DataEntered'] = pd.to_datetime(df['DataEntered'])
df = df[(df['DataEntered'].dt.year == 2024) & (df['DataEntered'].dt.month==3)]

In [ ]:

df = pd.merge(df, region_df[['ClientMan', 'Region']], left_on='ClientManager', right_on='ClientMan', how='left')

df = pd.merge(df, aksiya_df[['Goodid', 'Aksiya']], left_on='Goodid', right_on='Goodid', how='left')
df = pd.merge(df, paket_df[['Goodid', 'Paket']], left_on='Goodid', right_on='Goodid', how='left')


In [10]:
df.columns

Index(['Base', 'DocKind', 'InvoiceNumber', 'Goodid', 'GoodName', 'Producer',
       'Inn', 'ClientName', 'InvoiceManager', 'ClientManager', 'PaymentTerm',
       'BasePrice', 'SellingPrice', 'Quantity', 'DataEntered', 'BaseAmount',
       'TotalAmount', 'ClientMan', 'Region', 'Aksiya', 'Paket'],
      dtype='object')